In [ ]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.palettes import Dark2_8 as palette
from bokeh.plotting import figure

output_notebook()


df = pd.read_csv("log/server-stats-128.csv")
df

In [ ]:
p = figure(
    height=300,
    width=600,
    y_axis_label="Throughput",
    x_axis_label="Time [minutes]",
    tools=""
)
p.toolbar_location = None

# this will plot aggregate throughput of each model
# in your inference deployment (in units of seconds
# of data per second) as a function time. The aframe
# model, since it's the last, will naturally be the
# throughput that you observe. But you can look for
# disparities to get a sense of which models are
# acting as the bottleneck in your deployment.
t0  = df.timestamp.min()
for i, (model, subdf) in enumerate(df.groupby("model")):
    diff = subdf["timestamp"].diff()[1:]
    counts = subdf["count"].iloc[1:]
    inf_per_s = counts / diff
    s_per_s = inf_per_s * 128 / 4

    # you could also plot something similar by plotting
    # something like
    # queue = subdf["queue"] / subdf["count"]
    # which would give you some insights into how long
    # requests spent queuing for each model, which can
    # give you some insight into bottlenecks as well.
    t = (subdf["timestamp"].values[1:] - t0) / 60

    p.line(
        t,
        s_per_s,
        line_width=1.5,
        line_color=palette[i],
        line_alpha=0.7,
        legend_label=model
    )
show(p)